In [27]:
import pandas as pd
from os import path, curdir

ROOT = path.abspath(curdir)

datapath = path.join(ROOT, 'data', 'data.csv')
datapath2 = path.join(ROOT, 'data', 'calculations.csv')

df = pd.read_csv(datapath)
df2 = pd.read_csv(datapath2)
df = pd.merge(df, df2, how= 'left', on='State')
df = df.iloc[:,1:]
df = df.rename(columns={'RecidivismRate_x':'RecidivismRate'})
df=df.drop(columns=['Unnamed: 0_y','RecidivismRate_y'])
print(df.columns)

Index(['State', 'RecidivismRate', 'PrisonPopulation', 'ImprisonmentRate',
       'Population', 'qualityoflifetotalscore', 'QualityOfLifeAffordability',
       'QualityOfLifeEconomy', 'QualityOfLifeEducationAndHealth',
       'QualityOfLifeSafety', 'Male', 'Female', 'Total', 'Rape', 'Arson',
       'Fraud', 'Larceny', 'Robbery', 'Runaway', 'Weapons', 'Burglary',
       'Gambling', 'Vagrancy', 'Suspicion', 'Vandalism', 'Drunkenness',
       'Embezzlement', 'Sex_Offenses', 'Rape_Legacy', 'Simple_Assault',
       'Drug_Possession', 'Stolen_Property', 'Human_Trafficking',
       'Aggravated_Assault', 'All_Other_Offenses', 'Disorderly_Conduct',
       'Motor_Vehicle_Theft', 'Drug_Abuse_Violations', 'Liquor_Law_Violations',
       'Counterfeiting_or_Forgery', 'Drug_Sale_or_Manufacturing',
       'Drive_Under_the_Influence', 'Manslaughter_by_Negligence',
       'Murder_and_Nonnegligent_Homicide',
       'Curfew_and_Loitering_Law_Violations',
       'Prostitution_and_Commercialized_Vice',
     

In [28]:
from sklearn.model_selection import train_test_split
X = df[['PercentMale','PercentBlack','PercentWhite']]
Y = df['RecidivismRate']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8)


In [29]:
from sklearn import linear_model as lm
model = lm.LinearRegression()
model.fit(X_train,Y_train)
model.score(X_test, Y_test)

-0.40686290090083

In [30]:
from sklearn import tree
treeModel = tree.DecisionTreeRegressor()
treeModel.fit(X_train,Y_train)
treeModel.score(X_test,Y_test)

-0.6076114207440828

In [31]:
from sklearn.ensemble import RandomForestRegressor
treeModel = RandomForestRegressor()
treeModel.fit(X_train, Y_train)
treeModel.score(X_test,Y_test)

-0.0850833976336638

In [32]:
model = lm.Lasso(alpha=0.5)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)


-0.19288277784386065